## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
times = pd.read_csv('timesData.csv')
times.sort_values("year",ascending = False)
#Correggo errori di battitura notati risolvendo il punto 5
times['country'] = times['country'].str.replace('Unted Kingdom', 'United Kingdom')
times['country'] = times['country'].str.replace('Unisted States of America', 'United States of America')

In [3]:
#prova = times.groupby('university_name').size()
#df = pd.DataFrame(prova).reset_index()
#df.columns
#df.columns = ['university_name', 'number_of_year']

#m = pd.merge(times, df, how = 'left', on='university_name')
#times_with_no_single = m[m['number_of_year'] != 1]

In [4]:
times_most_recent = times.loc[times.groupby('university_name').idxmax()['year']]
times_least_recent = times.loc[times.groupby('university_name').idxmin()['year']]
times_least_recent

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405.0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501.0,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502.0,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166.0,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476.0,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41.0,42,École Normale Supérieure,France,66.8,44.9,48.2,95.7,30.7,68.6,"2,400",7.9,20%,46 : 54,2011
99.0,100,École Normale Supérieure de Lyon,France,51.1,37.6,34.4,88.8,26.1,57.0,"2,218",8.0,14%,49 : 51,2011
38.0,39,École Polytechnique,France,57.9,77.9,56.1,91.4,-,69.5,"2,429",4.8,30%,18 : 82,2011
47.0,48,École Polytechnique Fédérale de Lausanne,Switzerland,55.0,100.0,56.1,83.8,38.0,66.5,"9,666",10.5,54%,27 : 73,2011


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [5]:
times_most_recent['income'] = pd.to_numeric(times_most_recent['income'], errors = 'coerce') 
times_least_recent['income'] = pd.to_numeric(times_least_recent['income'], errors = 'coerce') 

In [6]:
times_most_recent.set_index('university_name', inplace = True)
times_least_recent.set_index('university_name', inplace = True)

In [7]:
income_diff = times_most_recent['income'] - times_least_recent['income']
income_diff = income_diff.reset_index()
income_diff.columns = ['university_name', 'difference']
income_diff.dropna()

,university_name,difference
1,Aalborg University,7.3
2,Aalto University,-0.3
3,Aarhus University,6.8
4,Aberystwyth University,-4.2
5,Adam Mickiewicz University,0.0
...,...,...
812,Zhejiang University,25.9
813,École Normale Supérieure,6.4
814,École Normale Supérieure de Lyon,5.6
816,École Polytechnique Fédérale de Lausanne,27.4


## 3. Find the university with the largest increase computed in the previous point

In [8]:
income_diff[income_diff['difference'] == income_diff['difference'].max()]

,university_name,difference
428,TU Dresden,67.8


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [9]:
times['num_students'] = pd.to_numeric(times['num_students'].str.replace(',', ''))

In [10]:
times_most_recent = times.loc[times.groupby('university_name').idxmax()['year']]

In [11]:
def fem_ratio(s):
    m = s.str.extract('\s*(?P<female>\d+)')
    fem = pd.to_numeric(m['female']) / 100
    return fem

In [12]:
fem = fem_ratio(times_most_recent['female_male_ratio'])
#times_most_recent['num_students']
times_most_recent['female'] = (times_most_recent['num_students'] * fem).round(0)
times_most_recent['male'] = times_most_recent['num_students'] - times_most_recent['female']

In [13]:
times_most_recent.groupby('country').sum()[['female', 'male']]

,female,male
country,,
Argentina,67191.0,41182.0
Australia,391736.0,321640.0
Austria,68364.0,66113.0
Bangladesh,21323.0,41393.0
Belarus,20219.0,9084.0
...,...,...
Uganda,18670.0,18670.0
Ukraine,17846.0,19250.0
United Arab Emirates,9516.0,4931.0


## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

In [14]:
times['fem_ratio'] = fem_ratio(times['female_male_ratio'])
times['male_ratio'] = 1 - times['fem_ratio']

In [15]:
times[times['male_ratio'] == 0].shape[0]

2

In [16]:
times2 = times[times['male_ratio'] != 0]
ratio = times2['fem_ratio'] / times2['male_ratio']
avg = ratio.mean()
avg

1.0684463755952236

In [17]:
times2['ratio'] = ratio

<ipython-input-17-6f48264178b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  times2['ratio'] = ratio


In [18]:
times3 = times2[times2['ratio'] < avg][['university_name', 'num_students', 'country', 'ratio']].drop_duplicates()
times3[['university_name', 'ratio']]

,university_name,ratio
1,California Institute of Technology,0.492537
2,Massachusetts Institute of Technology,0.587302
3,Stanford University,0.724138
4,Princeton University,0.818182
5,University of Cambridge,0.851852
...,...,...
2591,Warsaw University of Technology,0.515152
2597,Xidian University,0.408451
2598,Yeungnam University,0.923077
2599,Yıldız Technical University,0.562500


## 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.

In [19]:
times4 = times[['country', 'num_students']].drop_duplicates()

In [20]:
total_students = times4.groupby('country').sum()['num_students']
total_students.reset_index()

,country,num_students
0,Argentina,108373.0
1,Australia,743627.0
2,Austria,134477.0
3,Bangladesh,62716.0
4,Belarus,29303.0
...,...,...
65,Uganda,37340.0
66,Ukraine,37096.0
67,United Arab Emirates,14447.0
68,United Kingdom,1324842.0


In [23]:
times5 = times3.groupby('country').sum()['num_students'].reset_index()
times5.head()

,country,num_students
0,Australia,146235.0
1,Austria,61033.0
2,Bangladesh,62716.0
3,Brazil,278774.0
4,Canada,86779.0


In [24]:
merged = pd.merge(times5, total_students, on='country', suffixes=('_sel', '_tot'))
merged['ratio'] = (merged['num_students_sel'] / merged['num_students_tot']).round(2)
merged.head()

,country,num_students_sel,num_students_tot,ratio
0,Australia,146235.0,743627.0,0.20
1,Austria,61033.0,134477.0,0.45
2,Bangladesh,62716.0,62716.0,1.00
3,Brazil,278774.0,534688.0,0.52
4,Canada,86779.0,717054.0,0.12


## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [29]:
eas = pd.read_csv('educational_attainment_supplementary_data.csv')

In [30]:
eas.dropna(subset=['country_name', 'series_name'], inplace=True)

## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [32]:
anni = list(eas.columns[2:])

In [33]:
eas2 = eas.melt(id_vars=('country_name', 'series_name'), value_vars=anni, var_name='year',
                value_name='value').sort_values(by=['country_name', 'series_name']).dropna().reset_index(drop=True)
eas2

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1990,0.44000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1995,0.57000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2000,0.75000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2005,0.86000
...,...,...,...,...
318285,Zimbabwe,UIS: Percentage of population age 25+ with som...,2012,13.84323
318286,Zimbabwe,UIS: Percentage of population age 25+ with som...,2012,16.68491
318287,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2012,11.99412
318288,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2012,5.77150


## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

In [66]:
times2 = times['university_name'].drop_duplicates()
times2 = pd.DataFrame(times2.value_counts()).reset_index().rename(columns={"index": "university_name", "university_name": "count_times"})
times2

,university_name,count_times
0,Comenius University in Bratislava,1
1,Kumamoto University,1
2,Complutense University of Madrid,1
3,China University of Geosciences (Wuhan),1
4,University of Manchester,1
...,...,...
813,Kobe University,1
814,University of Rennes 1,1
815,University of Wollongong,1
816,Chinese University of Hong Kong,1


In [67]:
cwur2 = cwur['institution'].drop_duplicates()
cwur2 = pd.DataFrame(cwur2.value_counts()).reset_index().rename(columns={"index": "university_name", "institution": "count_cwur"})
cwur2

,university_name,count_cwur
0,Comenius University in Bratislava,1
1,University of Stirling,1
2,Shanghai Jiao Tong University,1
3,Nagoya City University,1
4,Yonsei University,1
...,...,...
1019,Osaka City University,1
1020,Hamamatsu University School of Medicine,1
1021,Gunma University,1
1022,University of Cagliari,1


In [68]:
shan2 = shan['university_name'].drop_duplicates()
shan2 = pd.DataFrame(shan2.value_counts()).reset_index().rename(columns={"index": "university_name", "university_name": "count_shan"})
shan2

,university_name,count_shan
0,The University of Texas Health Science Center ...,1
1,"Birkbeck, University of London",1
2,University of Helsinki,1
3,Hacettepe University,1
4,The Graduate University for Advanced Studies,1
...,...,...
653,Chiba University,1
654,University of Ulm,1
655,Kumamoto University,1
656,Northwestern University,1


In [69]:
tc = pd.merge(times2, cwur2, on='university_name', how='outer')

In [70]:
tcs = pd.merge(tc, shan2, on='university_name', how='outer').replace(np.NaN, 0)

In [71]:
tcs['somma'] = tcs['count_times'] + tcs['count_cwur'] + tcs['count_shan']

In [72]:
tcs[['university_name', 'somma']]

,university_name,somma
0,Comenius University in Bratislava,2.0
1,Kumamoto University,3.0
2,Complutense University of Madrid,3.0
3,China University of Geosciences (Wuhan),2.0
4,University of Manchester,2.0
...,...,...
1447,University of New Hampshire - Durham,1.0
1448,The University of Adelaide,1.0
1449,The University of Montana - Missoula,1.0
1450,University of Paris Diderot (Paris 7),1.0


## 11. In the times ranking, compute the number of times each university appears

In [57]:
times_count = times.groupby('university_name').count()['world_rank'].reset_index().rename(columns={"world_rank": "count_times"})
times_count

,university_name,count_times
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


## 12. Find the universities that appear at most twice in the times ranking

In [61]:
times12 = times.groupby('university_name').size()

In [62]:
times12[times.groupby('university_name').size()<3]

university_name
AGH University of Science and Technology    1
Adam Mickiewicz University                  1
Aix-Marseille University                    1
Ajou University                             1
Alexandru Ioan Cuza University              1
                                           ..
Yokohama City University                    1
Yokohama National University                1
Yuan Ze University                          2
Yıldız Technical University                 1
Örebro University                           1
Length: 408, dtype: int64

## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).